# Introduction

Create a binary classifier with `features_20240202_v1` which include competitor features and relative-to-competitor features. We include many features and some of them might not be important. Let's find the important features. Split models by surface type.

# Setup

In [1]:
import re
import tempfile
import warnings

import lightgbm as lgb
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import mlflow
import numpy as np
import pandas as pd
import seaborn as sns
from hyperopt import STATUS_OK, SparkTrials, Trials, fmin, hp, tpe
from hyperopt.pyll.base import scope
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import ColSpec, Schema
from pyspark.sql import SparkSession
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (
    accuracy_score,
    auc,
    confusion_matrix,
    f1_score,
    log_loss,
    precision_score,
    recall_score,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from JapanHorseRaceAnalytics.models.features_20240202_v1 import Features
from JapanHorseRaceAnalytics.utilities.base import get_base_dir, get_data_dir
from JapanHorseRaceAnalytics.utilities.metrics import (
    calculate_binary_classifier_statistics,
)
from JapanHorseRaceAnalytics.utilities.mlflow import get_colspecs
from JapanHorseRaceAnalytics.utilities.structured_logger import logger


pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

# Data Collection

In [2]:
warehouse_dir = f"{get_base_dir()}/spark-warehouse"
postgres_driver_path = f"{get_base_dir()}/jars/postgresql-42.7.1.jar"

spark = (
    SparkSession.builder.appName("20240211_competitors")
    .config("spark.driver.memory", "21g")
    .config("spark.driver.maxResultSize", "5g")
    .config("spark.sql.warehouse.dir", warehouse_dir)
    .config("spark.jars", postgres_driver_path)
    .config("spark.executor.extraClassPath", postgres_driver_path)
    .config("spark.driver.extraClassPath", postgres_driver_path)
    .enableHiveSupport()
    .getOrCreate()
)

24/02/17 15:04:25 WARN Utils: Your hostname, Hanks-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.40.105 instead (on interface en0)
24/02/17 15:04:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/02/17 15:04:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/17 15:04:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/02/17 15:04:26 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
def read_hive_table(
    table_name: str,
    schema: str,
    spark_session: SparkSession,
    use_cache: bool = True,
):
    save_path = get_data_dir() / "sql_tables" / f"{table_name}.snappy.parquet"
    if use_cache and save_path.exists():
        logger.info(f"Read from parquet {save_path} to pandas")
        return pd.read_parquet(save_path)
    logger.info(f"Read from hive {schema}.{table_name}")
    spark_df = spark_session.read.table(f"{schema}.{table_name}")
    logger.info(f"Write to parquet {save_path}")
    spark_df.write.mode("overwrite").parquet(str(save_path))
    logger.info(f"Read from parquet {save_path} to pandas")
    return pd.read_parquet(save_path)


data = read_hive_table(
    table_name="features_20240202_v1",
    schema="jhra_curated",
    spark_session=spark,
    use_cache=True,
).astype(Features.get_pandas_dtypes())

data.head()

{"event": "Read from parquet /Users/hankehly/Projects/JapanHorseRaceAnalytics/data/sql_tables/features_20240202_v1.snappy.parquet to pandas", "level": "info", "timestamp": "2024-02-17T06:04:26.521843Z", "logger": "__main__"}


,レースキー,馬番,血統登録番号,騎手コード,発走日時,調教師コード,先読み注意_着順,先読み注意_本賞金,先読み注意_単勝的中,先読み注意_単勝払戻金,先読み注意_複勝的中,先読み注意_複勝払戻金,場コード,四半期,距離,馬場状態コード,レース条件_トラック情報_右左,レース条件_トラック情報_内外,レース条件_種別,レース条件_条件,レース条件_記号,レース条件_重量,レース条件_グレード,頭数,トラック種別,馬場差,馬場状態内,馬場状態中,馬場状態外,直線馬場差最内,直線馬場差内,直線馬場差中,直線馬場差外,直線馬場差大外,芝種類,草丈,転圧,凍結防止剤,中間降水量,天候コード,性別,トラック種別瞬発戦好走馬,トラック種別消耗戦好走馬,一走前着順,二走前着順,三走前着順,四走前着順,五走前着順,六走前着順,ＩＤＭ,...,競争相手平均調教師1位完走差,競争相手平均調教師トップ3完走差,競争相手平均調教師1位完走率差,競争相手平均調教師トップ3完走率差,競争相手平均調教師場所レース数差,競争相手平均調教師場所1位完走差,競争相手平均調教師場所トップ3完走差,競争相手平均調教師場所1位完走率差,競争相手平均調教師場所トップ3完走率差,競争相手平均調教師本賞金累計差,競争相手平均調教師1位完走平均賞金差,競争相手平均調教師レース数平均賞金差,馬騎手レース数,馬騎手1位完走,馬騎手1位完走率,馬騎手トップ3完走,馬騎手トップ3完走率,馬騎手初二走,馬騎手同騎手,馬騎手場所レース数,馬騎手場所1位完走,馬騎手場所1位完走率,馬騎手場所トップ3完走,馬騎手場所トップ3完走率,馬調教師レース数,馬調教師1位完走,馬調教師1位完走率,馬調教師トップ3完走,馬調教師トップ3完走率,馬調教師初二走,馬調教師同調教師,馬調教師場所レース数,馬調教師場所1位完走,馬調教師場所1位完走率,馬調教師場所トップ3完走,馬調教師場所トップ3完走率,temperature,precipitation,snowfall,snow_depth,wind_speed,wind_direction,solar_radiation,local_air_pressure,sea_level_air_pressure,relative_humidity,vapor_pressure,dew_point_temperature,weather,visibility
0,01011101,04,98100695,10449,2001-08-04 00:50:00,10206,2,200.0,False,0,True,190,01,3,1000,30,1,1,12,A3,002,3,NaN,12,ダート,<NA>,1,1,1,1,1,0,0,0,NaN,<NA>,False,False,<NA>,2,牝,False,False,5,9,<NA>,<NA>,<NA>,<NA>,21.0,...,5.636364,-2.181818,-0.003834,-0.043248,0.363636,0.090909,-2.454545,0.02494,-0.06504,9512.318182,115.85049,-10.391494,0,0,0.0,0,0.0,True,False,0,0,0.0,0,0.0,2,0,0.0,0,0.0,False,True,0,0,0.0,0,0.0,21.933333,0.0,<NA>,0.0,4.433333,北西,2.098333,1011.15,1014.15,68.5,18.016667,15.866667,4,10.0
1,01011103,04,98102049,10356,2001-08-04 01:45:00,10263,6,0.0,False,0,False,0,01,3,1200,20,1,1,12,A3,102,3,NaN,16,芝,<NA>,1,1,1,1,1,0,0,0,NaN,<NA>,False,False,<NA>,1,牡,True,False,7,2,7,<NA>,<NA>,<NA>,36.0,...,-3.133333,-5.066667,-0.016287,-0.036119,-6.733333,-1.066667,-0.733333,-0.010847,0.12559,-18662.266667,-159.225818,-51.509013,2,0,0.0,0,0.0,False,False,0,0,0.0,0,0.0,9,0,0.0,1,0.111111,False,True,0,0,0.0,0,0.0,22.8,0.0,<NA>,0.0,3.9,北西,2.93,1010.95,1013.95,60.75,16.875,14.85,NaN,<NA>
2,01011103,09,98102902,10366,2001-08-04 01:45:00,10305,2,200.0,False,0,True,120,01,3,1200,20,1,1,12,A3,102,3,NaN,16,芝,<NA>,1,1,1,1,1,0,0,0,NaN,<NA>,False,False,<NA>,1,牡,True,False,3,2,<NA>,<NA>,<NA>,<NA>,38.0,...,-10.6,-20.0,-0.031598,-0.06815,-9.933333,-1.066667,-1.8,0.018783,0.12559,-28766.8,-206.365603,-72.468494,3,0,0.0,2,0.666667,False,True,0,0,0.0,0,0.0,8,0,0.0,3,0.375,False,True,0,0,0.0,0,0.0,22.8,0.0,<NA>,0.0,3.9,北西,2.93,1010.95,1013.95,60.75,16.875,14.85,NaN,<NA>
3,01011204,14,98110058,10392,2001-08-05 02:15:00,10219,6,0.0,False,0,False,0,01,3,1800,10,1,1,12,A3,102,3,NaN,14,芝,<NA>,1,1,1,1,1,0,0,0,NaN,<NA>,False,False,<NA>,1,牡,False,False,8,<NA>,<NA>,<NA>,<NA>,<NA>,33.4,...,13.615385,35.153846,0.021887,0.049687,15.769231,6.846154,9.923077,0.137634,0.14995,49377.5,316.358601,86.088353,4,0,0.0,0,0.0,False,False,0,0,0.0,0,0.0,10,0,0.0,1,0.1,False,True,0,0,0.0,0,0.0,22.3,0.0,<NA>,0.0,4.625,北北西,3.14,1010.325,1013.325,64.0,17.225,15.15,1,30.0
4,01011303,06,98103267,10412,2001-08-11 01:45:00,10303,3,130.0,False,0,True,1090,01,3,1700,10,1,1,12,A3,002,3,NaN,13,ダート,<NA>,1,1,1,1,1,0,0,0,NaN,<NA>,False,False,<NA>,2,牡,False,False,12,13,7,<NA>,<NA>,<NA>,17.0,...,13.75,21.75,0.033582,0.063804,7.75,1.333333,7.916667,0.039064,0.115922,-1223.125,-199.919166,12.856413,1,0,0.0,0,0.0,True,True,1,0,0.0,0,0.0,5,0,0.0,0,0.0,False,True,1,0,0.0,0,0.0,23.475,0.0,<NA>,0.0,0.825,北,1.435,1009.925,1012.925,65.0,18.725,16.475,NaN,<NA>


# Model Training

## Split Train and Test Data

In [4]:
X = data
y = data[Features.get_label()]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

X_train: (886532, 813)
X_test: (221634, 813)
y_train: (886532,)
y_test: (221634,)


## Define Objective Function

In [5]:
def create_objective_fn(
    X_train: pd.DataFrame,
    y_train: pd.Series,
    X_test: pd.DataFrame,
    y_test: pd.Series,
    df_payout: pd.DataFrame,
    experiment_name: str,
):
    """
    df_payout should have the same index as *_test and have the following columns:
    * レースキー
    * 馬番
    * 距離
    * 発走日時
    * 年齢
    * レース条件_グレード
    * 場コード
    * payout - amount won if betting 100 yen.
    """

    def train(params):
        def profit_loss(row, payout_column_name, bet_amount=100):
            if row["pred"] and row["actual"]:
                payout = row[payout_column_name] * (bet_amount / 100)
                return payout - bet_amount
            elif row["pred"] and not row["actual"]:
                return -bet_amount
            else:
                return 0

        mlflow.set_experiment(experiment_name=experiment_name)
        with mlflow.start_run():
            mlflow.log_params(params)

            numeric_features = X_train.select_dtypes("number").columns.tolist()
            categorical_features = X_train.select_dtypes("category").columns.tolist()
            preprocessor = ColumnTransformer(
                transformers=[
                    ("num", StandardScaler(), numeric_features),
                    (
                        "cat",
                        OneHotEncoder(handle_unknown="ignore"),
                        categorical_features,
                    ),
                ]
            )
            model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", lgb.LGBMClassifier(**params)),
                ]
            )

            # RestException: INVALID_PARAMETER_VALUE: Dataset schema exceeds the maximum length of 65535
            # Xy_train = pd.concat((X_train, y_train), axis=1)
            # dataset = mlflow.data.from_pandas(Xy_train, targets=y_train.name)
            # mlflow.log_input(dataset, context="train")

            input_schema = Schema(get_colspecs(X_train))
            output_schema = Schema([ColSpec("double", y_train.name)])
            signature = ModelSignature(inputs=input_schema, outputs=output_schema)
            input_example = X_train.iloc[:25]
            model.fit(X_train, y_train)
            mlflow.sklearn.log_model(
                sk_model=model,
                signature=signature,
                input_example=input_example,
                artifact_path="model",
            )

            y_pred_proba = model.predict_proba(X_test)[:, 1]
            y_pred = model.predict(X_test)

            metrics = {
                "loss": log_loss(y_test, y_pred_proba),
                "accuracy": accuracy_score(y_test, y_pred),
                "precision": precision_score(y_test, y_pred),
                "recall": recall_score(y_test, y_pred),
                "f1": f1_score(y_test, y_pred),
                "roc_auc": roc_auc_score(y_test, y_pred),
            }
            mlflow.log_metrics(metrics)

            # Calculate payout rates by group
            results = pd.concat(
                [
                    df_payout,
                    pd.DataFrame(
                        np.c_[y_test, y_pred, y_pred_proba],
                        columns=["actual", "pred", "pred_proba_true"],
                    ),
                ],
                axis=1,
            )
            payout_all = calculate_binary_classifier_statistics(
                results, group_by=None, payout_column_name="payout"
            )
            payout_month = calculate_binary_classifier_statistics(
                results,
                group_by=results["発走日時"].dt.month,
                payout_column_name="payout",
            )
            payout_distance = calculate_binary_classifier_statistics(
                results,
                group_by=pd.cut(results["距離"], bins=[0, 1400, 1800, 10000]),
                payout_column_name="payout",
            )
            payout_season = calculate_binary_classifier_statistics(
                results,
                group_by=results["発走日時"].dt.month % 12 // 3,
                payout_column_name="payout",
            )
            payout_year = calculate_binary_classifier_statistics(
                results,
                group_by=results["発走日時"].dt.year,
                payout_column_name="payout",
            )
            payout_age = calculate_binary_classifier_statistics(
                results,
                group_by=pd.cut(results["年齢"], bins=[0, 3, 6, 100]),
                payout_column_name="payout",
            )
            payout_grade = calculate_binary_classifier_statistics(
                results,
                group_by=results["レース条件_グレード"],
                payout_column_name="payout",
            )
            payout_racetrack = calculate_binary_classifier_statistics(
                results, group_by=results["場コード"], payout_column_name="payout"
            )
            payout = (
                pd.concat(
                    [
                        pd.DataFrame(payout_all).T.assign(group="all"),
                        pd.DataFrame(payout_month).T.assign(group="month"),
                        pd.DataFrame(payout_distance).T.assign(group="distance"),
                        pd.DataFrame(payout_season).T.assign(group="season"),
                        pd.DataFrame(payout_year).T.assign(group="year"),
                        pd.DataFrame(payout_age).T.assign(group="horse_age"),
                        pd.DataFrame(payout_grade).T.assign(group="grade"),
                        pd.DataFrame(payout_racetrack).T.assign(group="racetrack"),
                    ],
                    axis=0,
                )
                .rename_axis(index="part")
                .reset_index()
            )
            # Move "group" and "part" columns to the first position in this dataframe
            payout = payout[
                ["group", "part"]
                + [c for c in payout.columns if c not in ["group", "part"]]
            ]

            # Save payout rates as csv
            with tempfile.NamedTemporaryFile(prefix="payout_rate_", suffix=".csv") as f:
                payout.to_csv(f.name, index=False)
                mlflow.log_artifact(f.name)

            # Log payout rates as metrics
            payout_metrics = {}
            for group_name, group in payout.groupby("group"):
                for i, row in group.iterrows():
                    key = re.sub(r"\W", "_", f"payout_rate_{group_name}_{row['part']}")
                    payout_metrics[key] = row["payout_rate"]
            mlflow.log_metrics(payout_metrics)

            # Suppress UserWarning messages from matplotlib
            warnings.filterwarnings("ignore", category=UserWarning)

            # Plot payout rates by group
            sns.set_theme(style="whitegrid")
            fig, axes = plt.subplots(2, 4, figsize=(20, 10))
            for (group, df), ax in zip(payout.groupby("group"), axes.flatten()):
                sns.barplot(x="part", y="payout_rate", data=df, ax=ax)
                ax.set_title(group)
                ax.set_ylim(0, 150)
                ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
                ax.set_ylabel("payout rate")
                ax.set_xlabel("")
                ax.yaxis.set_major_formatter(ticker.PercentFormatter())
            with tempfile.NamedTemporaryFile(prefix="payout_rate_", suffix=".png") as f:
                plt.tight_layout()
                plt.savefig(f.name)
                plt.close()
                mlflow.log_artifact(f.name)

            # Plot bank balance over time
            results["profit_loss"] = results.apply(
                profit_loss, args=("payout", 100), axis=1
            )
            daily_profit_loss = results.groupby("発走日時")["profit_loss"].sum()
            bank_balance = daily_profit_loss.cumsum()
            plt.figure(figsize=(10, 10))
            ax = plt.subplot(1, 1, 1)
            ax.plot(bank_balance.index, bank_balance.values)
            ax.set_title("Bank Balance")
            ax.set_xlabel("Date")
            ax.set_ylabel("Bank Balance")
            ax.grid(True)
            ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:,.0f}"))
            with tempfile.NamedTemporaryFile(
                prefix="bank_balance_", suffix=".png"
            ) as f:
                plt.tight_layout()
                plt.savefig(f.name)
                plt.close()
                mlflow.log_artifact(f.name)

            # Confusion Matrix
            conf_matrix = confusion_matrix(y_test, y_pred)
            _, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
            sns.heatmap(conf_matrix, annot=True, fmt="g", cmap="Blues", ax=ax1)
            ax1.set_xlabel("Predicted")
            ax1.set_ylabel("Actual")
            ax1.set_title("Confusion Matrix")
            sns.heatmap(
                conf_matrix / conf_matrix.sum(axis=1)[:, None],
                annot=True,
                fmt=".2%",
                cmap="Blues",
                ax=ax2,
            )
            ax2.set_xlabel("Predicted")
            ax2.set_ylabel("Actual")
            ax2.set_title("Normalized Confusion Matrix")
            with tempfile.NamedTemporaryFile(
                prefix="confusion_matrix_", suffix=".png"
            ) as f:
                plt.tight_layout()
                plt.savefig(f.name)
                plt.close()
                mlflow.log_artifact(f.name)

            # ROC Curve
            fpr, tpr, _ = roc_curve(y_test, y_pred)
            roc_auc = auc(fpr, tpr)
            _, ax = plt.subplots(figsize=(10, 10))
            ax.plot(
                fpr,
                tpr,
                color="darkorange",
                lw=2,
                label="ROC curve (area = %0.2f)" % roc_auc,
            )
            ax.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
            ax.set_xlim([0.0, 1.0])
            ax.set_ylim([0.0, 1.0])
            ax.set_xlabel("False Positive Rate")
            ax.set_ylabel("True Positive Rate")
            ax.set_title("Receiver Operating Characteristic")
            ax.legend(loc="lower right")
            with tempfile.NamedTemporaryFile(prefix="roc_curve_", suffix=".png") as f:
                plt.tight_layout()
                plt.savefig(f.name)
                plt.close()
                mlflow.log_artifact(f.name)

            # Feature Importances
            feature_importances = zip(
                model.named_steps["preprocessor"].get_feature_names_out(),
                model.named_steps["classifier"].feature_importances_,
            )
            feature_importances_df = (
                pd.DataFrame(
                    data=feature_importances, columns=["feature", "importance"]
                )
                .sort_values("importance", ascending=False)
                .reset_index(drop=True)
            )
            with tempfile.NamedTemporaryFile(
                prefix="feature_importance_", suffix=".csv"
            ) as f:
                feature_importances_df.to_csv(f.name, index=False)
                mlflow.log_artifact(f.name)

            font_properties = fm.FontProperties(
                fname="/System/Library/Fonts/ヒラギノ角ゴシック W3.ttc"
            )
            sns.set_theme(style="whitegrid")
            plt.figure(figsize=(10, 12))
            ax = sns.barplot(
                x="importance", y="feature", data=feature_importances_df.iloc[:50]
            )
            ax.set_title("Feature Importances (Top 50)", fontproperties=font_properties)
            ax.set_xlabel("Importance", fontproperties=font_properties)
            ax.set_ylabel("Features", fontproperties=font_properties)
            for label in ax.get_yticklabels():
                label.set_fontproperties(font_properties)
            with tempfile.NamedTemporaryFile(
                prefix="feature_importance_", suffix=".png"
            ) as f:
                plt.tight_layout()
                plt.savefig(f.name)
                plt.close()
                mlflow.log_artifact(f.name)

            return {"status": STATUS_OK, "params": params, "model": model, **metrics}

    return train

## Hyperparameter Optimization

In [6]:
space = {
    "boosting_type": hp.choice("boosting_type", ["gbdt", "dart", "goss"]),
    "learning_rate": hp.loguniform("learning_rate", -5, 0),  # between e^-5 and 1
    "n_estimators": scope.int(hp.quniform("n_estimators", 100, 1000, 1)),
    "max_depth": scope.int(hp.quniform("max_depth", 3, 10, 1)),
    "num_leaves": scope.int(hp.quniform("num_leaves", 20, 150, 1)),
    "min_child_samples": scope.int(hp.quniform("min_child_samples", 20, 500, 1)),
    "feature_fraction": hp.uniform("feature_fraction", 0.5, 1.0),
    "lambda_l1": hp.uniform("lambda_l1", 0, 5),
    "lambda_l2": hp.uniform("lambda_l2", 0, 5),
    "min_split_gain": hp.uniform("min_split_gain", 0, 1),
    "min_child_weight": hp.uniform("min_child_weight", 0.001, 10),
    "subsample": hp.uniform("subsample", 0.5, 1),
    "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1),
    "reg_alpha": hp.uniform("reg_alpha", 0.0, 1.0),
    "reg_lambda": hp.uniform("reg_lambda", 0.0, 1.0),
    "objective": "binary",
    "class_weight": "balanced",
    "verbose": -1,
    "seed": 80,
}

### Turf Model

In [7]:
meta_feature_names = Features.get_feature_names_by_tag("metadata")
mask_train_turf = X_train["トラック種別"] == "芝"
mask_test_turf = X_test["トラック種別"] == "芝"

X_train_turf = X_train[mask_train_turf].drop(meta_feature_names, axis="columns")
y_train_turf = y_train[mask_train_turf]
X_test_turf = X_test[mask_test_turf].drop(meta_feature_names, axis="columns")
y_test_turf = y_test[mask_test_turf]

df_payout_columns = [
    "レースキー",
    "馬番",
    "距離",
    "発走日時",
    "先読み注意_複勝払戻金",
    "年齢",
    "レース条件_グレード",
    "場コード",
]

df_payout_turf = (
    data.iloc[X_test_turf.index]
    .reset_index(drop=True)[df_payout_columns]
    .rename(columns={"先読み注意_複勝払戻金": "payout"})
)

experiment_name_turf = "20240211_competitors__turf"
if mlflow.get_experiment_by_name(experiment_name_turf) is None:
    mlflow.create_experiment(experiment_name_turf)

fn_turf = create_objective_fn(
    X_train_turf,
    y_train_turf,
    X_test_turf,
    y_test_turf,
    df_payout=df_payout_turf,
    experiment_name=experiment_name_turf,
)

In [8]:
trials_turf = SparkTrials(parallelism=3, spark_session=spark)
fmin(fn=fn_turf, space=space, algo=tpe.suggest, max_evals=60, trials=trials_turf)

  0%|          | 0/60 [00:00<?, ?trial/s, best loss=?]

build_posterior_wrapper took 0.002353 seconds
TPE using 0 trials
build_posterior_wrapper took 0.002181 seconds
TPE using 1/1 trials with best loss inf
build_posterior_wrapper took 0.001765 seconds
TPE using 2/2 trials with best loss inf
build_posterior_wrapper took 0.001744 seconds
TPE using 3/3 trials with best loss inf
2024/02/15 08:23:16 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed. The model artifacts have been logged successfully under /Users/hankehly/Projects/JapanHorseRaceAnalytics/mlruns/17/174672fa671040c6a3d21ad6cc0a2987/artifacts. Set logging level to DEBUG via `logging.getLogger("mlflow").setLevel(logging.DEBUG)` to see the full traceback.
Closing down clientserver connection                                            


  2%|▏         | 1/60 [03:31<3:27:47, 211.31s/trial, best loss: 0.5197401275715573]

build_posterior_wrapper took 0.003196 secondse 2:>                  (0 + 1) / 1]
TPE using 4/4 trials with best loss 0.519740
2024/02/15 08:24:12 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed. The model artifacts have been logged successfully under /Users/hankehly/Projects/JapanHorseRaceAnalytics/mlruns/17/a398ff606eb24976b618ae39b58a1090/artifacts. Set logging level to DEBUG via `logging.getLogger("mlflow").setLevel(logging.DEBUG)` to see the full traceback.
Closing down clientserver connection                                            


  3%|▎         | 2/60 [04:26<1:55:41, 119.67s/trial, best loss: 0.5197401275715573]

...

2024/02/15 10:28:16 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed. The model artifacts have been logged successfully under /Users/hankehly/Projects/JapanHorseRaceAnalytics/mlruns/17/02355db18327461b9f72b34135ebdc81/artifacts. Set logging level to DEBUG via `logging.getLogger("mlflow").setLevel(logging.DEBUG)` to see the full traceback.
Closing down clientserver connection                                            


 98%|█████████▊| 59/60 [2:08:26<02:10, 130.62s/trial, best loss: 0.47036847645210006]

Queue empty, exiting run.
Closing down clientserver connection
Total Trials: 60: 59 succeeded, 1 failed, 0 cancelled.


Total Trials: 60: 59 succeeded, 1 failed, 0 cancelled.


{'boosting_type': 0,
 'colsample_bytree': 0.9480736601486514,
 'feature_fraction': 0.9412315757597349,
 'lambda_l1': 2.20456622078147,
 'lambda_l2': 4.933460303922979,
 'learning_rate': 0.09718115779176308,
 'max_depth': 10.0,
 'min_child_samples': 33.0,
 'min_child_weight': 6.260291002769891,
 'min_split_gain': 0.09603499236876085,
 'n_estimators': 766.0,
 'num_leaves': 139.0,
 'reg_alpha': 0.9992796927571433,
 'reg_lambda': 0.926255887431294,
 'subsample': 0.8079044658652886}

### Dirt Model

In [8]:
meta_feature_names = Features.get_feature_names_by_tag("metadata")
mask_train_dirt = X_train["トラック種別"] == "ダート"
mask_test_dirt = X_test["トラック種別"] == "ダート"

X_train_dirt = X_train[mask_train_dirt].drop(meta_feature_names, axis="columns")
y_train_dirt = y_train[mask_train_dirt]
X_test_dirt = X_test[mask_test_dirt].drop(meta_feature_names, axis="columns")
y_test_dirt = y_test[mask_test_dirt]

df_payout_columns = [
    "レースキー",
    "馬番",
    "距離",
    "発走日時",
    "先読み注意_複勝払戻金",
    "年齢",
    "レース条件_グレード",
    "場コード",
]

df_payout_dirt = (
    data.iloc[X_test_dirt.index]
    .reset_index(drop=True)[df_payout_columns]
    .rename(columns={"先読み注意_複勝払戻金": "payout"})
)

experiment_name_dirt = "20240211_competitors__dirt"
if mlflow.get_experiment_by_name(experiment_name_dirt) is None:
    mlflow.create_experiment(experiment_name_dirt)

fn_dirt = create_objective_fn(
    X_train_dirt,
    y_train_dirt,
    X_test_dirt,
    y_test_dirt,
    df_payout=df_payout_dirt,
    experiment_name=experiment_name_dirt,
)

In [9]:
trials_dirt = SparkTrials(parallelism=3, spark_session=spark)
fmin(fn=fn_dirt, space=space, algo=tpe.suggest, max_evals=60, trials=trials_dirt)

  0%|          | 0/60 [00:00<?, ?trial/s, best loss=?]

build_posterior_wrapper took 0.002523 seconds
TPE using 0 trials
build_posterior_wrapper took 0.001818 seconds
TPE using 1/1 trials with best loss inf
build_posterior_wrapper took 0.001757 seconds
TPE using 2/2 trials with best loss inf
build_posterior_wrapper took 0.003249 seconds
TPE using 3/3 trials with best loss inf
2024/02/15 10:50:55 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed. The model artifacts have been logged successfully under /Users/hankehly/Projects/JapanHorseRaceAnalytics/mlruns/20/4a8300d55bc2463d9fbd1e1917d9bfac/artifacts. Set logging level to DEBUG via `logging.getLogger("mlflow").setLevel(logging.DEBUG)` to see the full traceback.
Closing down clientserver connection                                            


  2%|▏         | 1/60 [03:16<3:13:10, 196.45s/trial, best loss: 0.5176572017074095]

build_posterior_wrapper took 0.004090 secondse 2:>                  (0 + 1) / 1]
TPE using 4/4 trials with best loss 0.517657
2024/02/15 10:53:29 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed. The model artifacts have been logged successfully under /Users/hankehly/Projects/JapanHorseRaceAnalytics/mlruns/20/f75e6ab2289446a4bae18585c5899c0a/artifacts. Set logging level to DEBUG via `logging.getLogger("mlflow").setLevel(logging.DEBUG)` to see the full traceback.
2024/02/15 10:53:37 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed. The model artifacts have been logged successfully under /Users/hankehly/Projects/JapanHorseRaceAnalytics/mlruns/20/d815c387c56648e78e0ac4f16ea066cc/artifacts. Set logging level to DEBUG via `logging.getLogger("mlflow").setLevel(logging.DEBUG)` to see the full traceback.
Closing down clientserver connection                                            


  3%|▎         | 2/60 [05:51<2:46:21, 172.09s/trial, best loss: 0.5101523097557341]

...

2024/02/15 13:04:48 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed. The model artifacts have been logged successfully under /Users/hankehly/Projects/JapanHorseRaceAnalytics/mlruns/20/6d2ff1cbdcd84d7d8636d527553ef6bb/artifacts. Set logging level to DEBUG via `logging.getLogger("mlflow").setLevel(logging.DEBUG)` to see the full traceback.
Closing down clientserver connection                                            


100%|██████████| 60/60 [2:17:08<00:00, 137.14s/trial, best loss: 0.47480602323486787]

Queue empty, exiting run.


Closing down clientserver connection
Total Trials: 60: 60 succeeded, 0 failed, 0 cancelled.
Total Trials: 60: 60 succeeded, 0 failed, 0 cancelled.


{'boosting_type': 1,
 'colsample_bytree': 0.7332050245764046,
 'feature_fraction': 0.7720929534556822,
 'lambda_l1': 2.323874238782759,
 'lambda_l2': 1.9333797615058455,
 'learning_rate': 0.3187890033154122,
 'max_depth': 10.0,
 'min_child_samples': 21.0,
 'min_child_weight': 6.04074523620894,
 'min_split_gain': 0.1844148611798127,
 'n_estimators': 623.0,
 'num_leaves': 146.0,
 'reg_alpha': 0.9068112839793774,
 'reg_lambda': 0.852068188453003,
 'subsample': 0.8098002257945933}

# Model Evaluation

# Conclusion